In [1]:
!pip install openpyxl

import openpyxl
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 25.2 MB/s eta 0:00:00


Matplotlib is building the font cache; this may take a moment.


# Législatives 2024 #

In [2]:
# Obtenez le chemin du répertoire actuel (qui est RisqueRNAnalyse)
current_dir = os.getcwd()

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(current_dir, 'Database', 'Legislatives2024CircoT2.xlsx')

# Chargez le fichier Excel dans un DataFrame
Legi2024CircoT2 = pd.read_excel(fichier_excel)

In [3]:
Legi2024CircoT2 = Legi2024CircoT2[['Code département', 'Code circonscription législative', '% Votants', '% Abstentions', 'Exprimés', 'Nuance candidat 1', '% Voix/inscrits 1', '% Voix/exprimés 1', 'Elu 1',  'Nuance candidat 2', '% Voix/inscrits 2', '% Voix/exprimés 2', 'Elu 2', 'Nuance candidat 3', '% Voix/inscrits 3', '% Voix/exprimés 3', 'Elu 3',  'Nuance candidat 4', '% Voix/inscrits 4', '% Voix/exprimés 4', 'Elu 4']]

In [4]:
Legi2024CircoT2.columns

Index(['Code département', 'Code circonscription législative', '% Votants',
       '% Abstentions', 'Exprimés', 'Nuance candidat 1', '% Voix/inscrits 1',
       '% Voix/exprimés 1', 'Elu 1', 'Nuance candidat 2', '% Voix/inscrits 2',
       '% Voix/exprimés 2', 'Elu 2', 'Nuance candidat 3', '% Voix/inscrits 3',
       '% Voix/exprimés 3', 'Elu 3', 'Nuance candidat 4', '% Voix/inscrits 4',
       '% Voix/exprimés 4', 'Elu 4'],
      dtype='object')

In [5]:
# Dictionnaire de correspondance des labels pour bord24
mapping_bord24 = {
    'ENS': 'ENS',
    'UG': 'UG',
    'RN': 'XD', #RN devient XD
    'LR': 'DVD', # LR dit DVD
    'DVD': 'DVD',   
    'UXD': 'XD',   # Regroupe UXD sous XD
    'DVG': 'DVG',
    'REG': 'REG',
    'HOR': 'ENS',  # Regroupe HOR sous ENS
    'DVC': 'DVC',
    'UDI': 'DVD',   # Regroupe UDI sous DVD
    'SOC': 'DVG',  # Regroupe SOC sous DVG
    'DIV': 'DIV',
    'ECO': 'DVG'   # Regroupe ECO sous DVG
}

# Liste des colonnes à analyser
columns_to_analyze = ['Nuance candidat 1', 'Nuance candidat 2', 'Nuance candidat 3', 'Nuance candidat 4']

# Remapper les valeurs des colonnes spécifiées
for column in columns_to_analyze:
    Legi2024CircoT2[column] = Legi2024CircoT2[column].map(mapping_bord24)

# Dictionnaire pour stocker les occurrences agrégées de chaque valeur unique après remapping
aggregated_counts = {}

# Compter les occurrences de chaque valeur dans chaque colonne et les agréger
for column in columns_to_analyze:
    value_counts = Legi2024CircoT2[column].value_counts()
    for value, count in value_counts.items():
        if value in aggregated_counts:
            aggregated_counts[value] += count
        else:
            aggregated_counts[value] = count

# Afficher les occurrences agrégées pour chaque valeur unique
for value, total in aggregated_counts.items():
    print(f"La somme des occurrences de la nuance '{value}' est : {total}")

La somme des occurrences de la nuance 'XD' est : 441
La somme des occurrences de la nuance 'UG' est : 281
La somme des occurrences de la nuance 'ENS' est : 227
La somme des occurrences de la nuance 'DVD' est : 99
La somme des occurrences de la nuance 'DVG' est : 20
La somme des occurrences de la nuance 'REG' est : 12
La somme des occurrences de la nuance 'DIV' est : 2
La somme des occurrences de la nuance 'DVC' est : 9


In [6]:
# Liste des colonnes de nuances et des colonnes d'élu
nuance_columns = ['Nuance candidat 1', 'Nuance candidat 2', 'Nuance candidat 3', 'Nuance candidat 4']
elu_columns = ['Elu 1', 'Elu 2', 'Elu 3', 'Elu 4']

# Fonction pour obtenir la nuance du candidat élu
def get_elected_nuance(row):
    for nuance_col, elu_col in zip(nuance_columns, elu_columns):
        if row[elu_col] == 'élu':
            return row[nuance_col]
    return None

# Ajouter une nouvelle colonne 'Nuance élu'
Legi2024CircoT2['Nuance élu'] = Legi2024CircoT2.apply(get_elected_nuance, axis=1)
Legi2024CircoT2

,Code département,Code circonscription législative,% Votants,% Abstentions,Exprimés,Nuance candidat 1,% Voix/inscrits 1,% Voix/exprimés 1,Elu 1,Nuance candidat 2,...,Elu 2,Nuance candidat 3,% Voix/inscrits 3,% Voix/exprimés 3,Elu 3,Nuance candidat 4,% Voix/inscrits 4,% Voix/exprimés 4,Elu 4,Nuance élu
0,01,0101,"71,74%","28,26%",60005,XD,"30,07%","43,52%",NaN,DVD,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DVD
1,01,0102,"72,20%","27,80%",70739,XD,"31,17%","44,91%",NaN,ENS,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
2,01,0103,"64,96%","35,04%",52317,ENS,"39,16%","63,00%",élu,XD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
3,01,0104,"70,46%","29,54%",64623,ENS,"32,70%","48,65%",NaN,XD,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XD
4,01,0105,"67,87%","32,13%",47004,UG,"25,62%","42,47%",NaN,XD,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,ZZ,ZZ07,"47,31%","52,69%",59300,ENS,"26,39%","58,21%",élu,UG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
497,ZZ,ZZ08,"24,32%","75,68%",34730,DVD,"11,03%","47,30%",NaN,ENS,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
498,ZZ,ZZ09,"28,55%","71,45%",35164,ENS,"6,82%","25,29%",NaN,UG,...,élu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UG
499,ZZ,ZZ10,"30,37%","69,63%",32041,ENS,"14,98%","53,23%",élu,UG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS


In [7]:
# Créer une copie de Legi2024CircoT2
copie = Legi2024CircoT2.copy()

# Remplacer les virgules par des points, supprimer les signes '%' et convertir les valeurs en numériques dans les colonnes pertinentes de copie
for col in copie.columns:
    if '% Voix/inscrits' in col or '% Voix/exprimés' in col:  # Vérifiez les colonnes de scores
        if copie[col].dtype == 'object':  # Vérifiez si la colonne est de type chaîne de caractères
            copie[col] = copie[col].str.replace('%', '')  # Supprimer le signe '%'
            copie[col] = copie[col].str.replace(',', '.')  # Remplacer les virgules par des points
            copie[col] = pd.to_numeric(copie[col], errors='coerce')  # Convertir en valeurs flottantes, en gérant les erreurs

# Réorganiser les colonnes par nuances
nuances = list(set(copie[nuance_columns].values.flatten()))

# Colonnes de scores à réorganiser par nuances
score_columns = ['% Voix/inscrits', '% Voix/exprimés']

# Initialiser un nouveau DataFrame avec les colonnes réorganisées
final_columns = ['Code département', 'Code circonscription législative', 'Nuance élu', '% Votants', '% Abstentions'] + [f'{score} {nuance}' for nuance in nuances for score in score_columns]
final_df = pd.DataFrame(columns=final_columns)

# Copier les colonnes 'code département', 'code circonscription législative' et 'Nuance élu' dans le nouveau DataFrame
final_df['Code département'] = copie['Code département']
final_df['Code circonscription législative'] = copie['Code circonscription législative']
final_df['Nuance élu'] = copie['Nuance élu']
final_df['% Votants'] = copie['% Votants']
final_df['% Abstentions'] = copie['% Abstentions']

# Remplir le DataFrame final avec les données des scores réorganisées par nuances
for nuance in nuances:
    for score_col in score_columns:
        score_data = []
        for i in range(1, 5):  # Assumant qu'il y a jusqu'à 'Nuance candidat 4' et '% Voix/inscrits 4', etc.
            nuance_col = f'Nuance candidat {i}'
            score_col_name = f'{score_col} {i}'
            if score_col_name in copie.columns:
                score_data.append(
                    copie.apply(
                        lambda row: row[score_col_name] if row[nuance_col] == nuance else None, axis=1
                    )
                )
        if score_data:
            # Combiner les scores des différentes colonnes, en s'assurant qu'il n'y a pas de NaN
            combined_scores = pd.concat(score_data, axis=1).max(axis=1, skipna=True)
            final_df[f'{score_col} {nuance}'] = combined_scores
        else:
            # Si aucune colonne de score trouvée, remplir avec des valeurs manquantes
            final_df[f'{score_col} {nuance}'] = None

final_df = final_df.drop(columns=['% Voix/inscrits nan', '% Voix/exprimés nan'])

final_df


,Code département,Code circonscription législative,Nuance élu,% Votants,% Abstentions,% Voix/inscrits REG,% Voix/exprimés REG,% Voix/inscrits DVG,% Voix/exprimés DVG,% Voix/inscrits DVC,...,% Voix/inscrits DIV,% Voix/exprimés DIV,% Voix/inscrits DVD,% Voix/exprimés DVD,% Voix/inscrits XD,% Voix/exprimés XD,% Voix/inscrits ENS,% Voix/exprimés ENS,% Voix/inscrits UG,% Voix/exprimés UG
0,01,0101,DVD,"71,74%","28,26%",None,None,None,None,None,...,None,None,39.02,56.48,30.07,43.52,NaN,NaN,None,None
1,01,0102,ENS,"72,20%","27,80%",None,None,None,None,None,...,None,None,None,None,31.17,44.91,38.24,55.09,None,None
2,01,0103,ENS,"64,96%","35,04%",None,None,None,None,None,...,None,None,None,None,23.0,37.0,39.16,63.00,None,None
3,01,0104,XD,"70,46%","29,54%",None,None,None,None,None,...,None,None,None,None,34.52,51.35,32.70,48.65,None,None
4,01,0105,XD,"67,87%","32,13%",None,None,None,None,None,...,None,None,None,None,34.7,57.53,NaN,NaN,25.62,42.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,ZZ,ZZ07,ENS,"47,31%","52,69%",None,None,None,None,None,...,None,None,None,None,None,None,26.39,58.21,18.95,41.79
497,ZZ,ZZ08,ENS,"24,32%","75,68%",None,None,None,None,None,...,None,None,11.03,47.3,None,None,12.29,52.70,None,None
498,ZZ,ZZ09,UG,"28,55%","71,45%",None,None,None,None,None,...,None,None,None,None,None,None,6.82,25.29,20.15,74.71
499,ZZ,ZZ10,ENS,"30,37%","69,63%",None,None,None,None,None,...,None,None,None,None,None,None,14.98,53.23,13.16,46.77


In [8]:
circo242 = final_df.copy()
circo242

,Code département,Code circonscription législative,Nuance élu,% Votants,% Abstentions,% Voix/inscrits REG,% Voix/exprimés REG,% Voix/inscrits DVG,% Voix/exprimés DVG,% Voix/inscrits DVC,...,% Voix/inscrits DIV,% Voix/exprimés DIV,% Voix/inscrits DVD,% Voix/exprimés DVD,% Voix/inscrits XD,% Voix/exprimés XD,% Voix/inscrits ENS,% Voix/exprimés ENS,% Voix/inscrits UG,% Voix/exprimés UG
0,01,0101,DVD,"71,74%","28,26%",None,None,None,None,None,...,None,None,39.02,56.48,30.07,43.52,NaN,NaN,None,None
1,01,0102,ENS,"72,20%","27,80%",None,None,None,None,None,...,None,None,None,None,31.17,44.91,38.24,55.09,None,None
2,01,0103,ENS,"64,96%","35,04%",None,None,None,None,None,...,None,None,None,None,23.0,37.0,39.16,63.00,None,None
3,01,0104,XD,"70,46%","29,54%",None,None,None,None,None,...,None,None,None,None,34.52,51.35,32.70,48.65,None,None
4,01,0105,XD,"67,87%","32,13%",None,None,None,None,None,...,None,None,None,None,34.7,57.53,NaN,NaN,25.62,42.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,ZZ,ZZ07,ENS,"47,31%","52,69%",None,None,None,None,None,...,None,None,None,None,None,None,26.39,58.21,18.95,41.79
497,ZZ,ZZ08,ENS,"24,32%","75,68%",None,None,None,None,None,...,None,None,11.03,47.3,None,None,12.29,52.70,None,None
498,ZZ,ZZ09,UG,"28,55%","71,45%",None,None,None,None,None,...,None,None,None,None,None,None,6.82,25.29,20.15,74.71
499,ZZ,ZZ10,ENS,"30,37%","69,63%",None,None,None,None,None,...,None,None,None,None,None,None,14.98,53.23,13.16,46.77


In [10]:
# Obtenez le chemin du répertoire actuel (qui est RisqueRNAnalyse)
current_dir = os.getcwd()

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(current_dir, 'Database', 'Legislatives2024CircoT1.xlsx')

# Chargez le fichier Excel dans un DataFrame
Legi2024CircoT1 = pd.read_excel(fichier_excel)

In [11]:
# Liste des colonnes de nuances et des colonnes d'élu
nuance_columns = ['Nuance candidat 1', 'Nuance candidat 2', 'Nuance candidat 3', 'Nuance candidat 4', 'Nuance candidat 5', 'Nuance candidat 6', 'Nuance candidat 7', 'Nuance candidat 8', 'Nuance candidat 9', 'Nuance candidat 10', 'Nuance candidat 11', 'Nuance candidat 12','Nuance candidat 13', 'Nuance candidat 14', 'Nuance candidat 15']
elu_columns = ['Elu 1', 'Elu 2', 'Elu 3', 'Elu 4', 'Elu 5','Elu 6', 'Elu 7', 'Elu 8','Elu 9', 'Elu 10', 'Elu 11', 'Elu 12', 'Elu 13', 'Elu 14', 'Elu 15']

# Fonction pour obtenir la nuance du candidat élu
def get_elected_nuance(row):
    for nuance_col, elu_col in zip(nuance_columns, elu_columns):
        if row[elu_col] == 'élu':
            return row[nuance_col]
    return None

Legi2024CircoT1['Nuance élu'] = Legi2024CircoT1.apply(get_elected_nuance, axis=1)


In [12]:
# Créer une copie de Legi2024CircoT1 pour le traitement
copie = Legi2024CircoT1.copy()

# Nettoyer et convertir les colonnes de scores dans copie
for col in copie.columns:
    if '% Voix/inscrits' in col or '% Voix/exprimés' in col:  # Vérifiez les colonnes de scores
        if copie[col].dtype == 'object':  # Vérifiez si la colonne est de type chaîne de caractères
            copie[col] = copie[col].str.replace('%', '')  # Supprimer le signe '%'
            copie[col] = copie[col].str.replace(',', '.')  # Remplacer les virgules par des points
            copie[col] = pd.to_numeric(copie[col], errors='coerce')  # Convertir en valeurs flottantes

# Supprimer les colonnes dont les valeurs dans '% Voix/inscrits' ou '% Voix/exprimés' sont inférieures à 5%
cols_to_drop = []
for i in range(1, 20):  # Pour les colonnes allant de 1 à 19
    num_col = f'Numéro de panneau {i}'
    nuance_col = f'Nuance candidat {i}'
    nom_col = f'Nom candidat {i}'
    prenom_col = f'Prénom candidat {i}'
    sexe_col = f'Sexe candidat {i}'
    voix_col = f'Voix {i}'
    voix_inscrits_col = f'% Voix/inscrits {i}'
    voix_exprimes_col = f'% Voix/exprimés {i}'
    elu_col = f'Elu {i}'

    # Vérifier les colonnes de scores pour déterminer si les valeurs sont inférieures à 5%
    if voix_inscrits_col in copie.columns and voix_exprimes_col in copie.columns:
        voix_inscrits_scores = copie[voix_inscrits_col].dropna()
        voix_exprimes_scores = copie[voix_exprimes_col].dropna()
        if (voix_inscrits_scores.max() < 5) or (voix_exprimes_scores.max() < 5):
            cols_to_drop.extend([num_col, nuance_col, nom_col, prenom_col, sexe_col, voix_col, voix_inscrits_col, voix_exprimes_col, elu_col])

# Supprimer les colonnes dans copie
copie = copie.drop(columns=cols_to_drop)

copie


,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Numéro de panneau 15,Nuance candidat 15,Nom candidat 15,Prénom candidat 15,Sexe candidat 15,Voix 15,% Voix/inscrits 15,% Voix/exprimés 15,Elu 15,Nuance élu
0,1,Ain,101,1ère circonscription,86843,61830,"71,20%",25013,"28,80%",60495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,1,Ain,102,2ème circonscription,101874,73437,"72,09%",28437,"27,91%",71918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,1,Ain,103,3ème circonscription,84130,54961,"65,33%",29169,"34,67%",53720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,1,Ain,104,4ème circonscription,96119,67580,"70,31%",28539,"29,69%",65687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,1,Ain,105,5ème circonscription,77900,52973,"68,00%",24927,"32,00%",51532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,Français établis hors de France,ZZ07,7ème circonscription,130824,58559,"44,76%",72265,"55,24%",58050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
573,ZZ,Français établis hors de France,ZZ08,8ème circonscription,148957,33070,"22,20%",115887,"77,80%",32482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
574,ZZ,Français établis hors de France,ZZ09,9ème circonscription,130387,36603,"28,07%",93784,"71,93%",35880,...,15.0,DIV,PERIMONY,Sébastien,MASCULIN,172.0,0.13,0.48,NaN,None
575,ZZ,Français établis hors de France,ZZ10,10ème circonscription,113855,36502,"32,06%",77353,"67,94%",35824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [13]:
# Créer une copie de votre DataFrame
circo241 = copie.copy()

# Liste des nuances spécifiques à utiliser
nuances = ['EXG', 'RN', 'UG', 'ENS', 'LR', 'REC', 'DIV', 'DVD', 'DVC', 'ECO', 'DVG', 'DSV', 'REG', 'UXD', 'EXD', 'UDI', 'HOR', 'SOC']

# Colonnes de scores à réorganiser par nuances
score_columns = ['% Voix/inscrits', '% Voix/exprimés']

# Nettoyer et convertir les colonnes de scores
for col in circo241.columns:
    if '% Voix/inscrits' in col or '% Voix/exprimés' in col:
        if circo241[col].dtype == 'object':
            circo241[col] = circo241[col].str.replace('%', '')  # Supprimer le signe '%'
            circo241[col] = circo241[col].str.replace(',', '.')  # Remplacer les virgules par des points
            circo241[col] = pd.to_numeric(circo241[col], errors='coerce')  # Convertir en valeurs flottantes

# Initialiser un DataFrame pour les données réorganisées
reorganized_columns = ['Code département', 'Code circonscription législative', 'Nuance élu'] + [f'{score} {nuance}' for nuance in nuances for score in score_columns]
reorganized_df = pd.DataFrame(columns=reorganized_columns)

# Copier les colonnes de base
reorganized_df['Code département'] = circo241['Code département']
reorganized_df['Code circonscription législative'] = circo241['Code circonscription législative']
reorganized_df['Nuance élu'] = circo241['Nuance élu']

# Remplir le DataFrame réorganisé avec les données des scores par nuance
for nuance in nuances:
    for score_col in score_columns:
        # Trouver les colonnes spécifiques aux scores
        score_data = []
        for i in range(1, 20):  # Assumer qu'il y a jusqu'à 'Nuance candidat 19'
            nuance_col = f'Nuance candidat {i}'
            score_col_name = f'{score_col} {i}'
            if score_col_name in circo241.columns:
                score_data.append(
                    circo241.apply(
                        lambda row: row[score_col_name] if row[nuance_col] == nuance else None, axis=1
                    )
                )
        if score_data:
            # Combiner les scores des différentes colonnes
            combined_scores = pd.concat(score_data, axis=1).max(axis=1, skipna=True)
            reorganized_df[f'{score_col} {nuance}'] = combined_scores
        else:
            # Si aucune colonne de score trouvée, remplir avec des valeurs manquantes
            reorganized_df[f'{score_col} {nuance}'] = None

reorganized_df


,Code département,Code circonscription législative,Nuance élu,% Voix/inscrits EXG,% Voix/exprimés EXG,% Voix/inscrits RN,% Voix/exprimés RN,% Voix/inscrits UG,% Voix/exprimés UG,% Voix/inscrits ENS,...,% Voix/inscrits UXD,% Voix/exprimés UXD,% Voix/inscrits EXD,% Voix/exprimés EXD,% Voix/inscrits UDI,% Voix/exprimés UDI,% Voix/inscrits HOR,% Voix/exprimés HOR,% Voix/inscrits SOC,% Voix/exprimés SOC
0,1,101,None,0.48,0.69,27.43,39.37,16.34,23.45,8.13,...,None,None,None,None,None,None,None,None,None,None
1,1,102,None,0.72,1.02,27.67,39.2,16.67,23.61,17.09,...,None,None,1.83,2.59,None,None,None,None,None,None
2,1,103,None,0.54,0.85,20.51,32.11,16.04,25.12,20.71,...,None,None,None,None,None,None,None,None,None,None
3,1,104,None,0.73,1.07,31.44,46.01,13.64,19.96,14.95,...,None,None,None,None,None,None,None,None,None,None
4,1,105,None,0.78,1.18,None,None,16.10,24.34,7.75,...,25.88,39.12,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,ZZ07,None,None,None,3.46,7.79,14.46,32.58,16.76,...,None,None,None,None,None,None,None,None,None,None
573,ZZ,ZZ08,None,None,None,None,None,5.05,23.14,5.27,...,None,None,None,None,None,None,None,None,None,None
574,ZZ,ZZ09,None,None,None,None,None,14.19,51.57,4.32,...,None,None,None,None,None,None,None,None,None,None
575,ZZ,ZZ10,None,None,None,None,None,10.23,32.52,10.01,...,5.4,17.16,None,None,None,None,None,None,None,None


In [14]:
final_df = reorganized_df.copy()
# Conserver les colonnes essentielles
essential_columns = ['Code département', 'Code circonscription législative', 'Nuance élu']

# Liste des colonnes à vérifier (toutes sauf les colonnes essentielles)
columns_to_check = [col for col in final_df.columns if col not in essential_columns]

# Conserver les colonnes qui contiennent au moins une valeur <= 5
valid_columns = [col for col in columns_to_check if (final_df[col] >= 5).any()]

# Conserver les colonnes essentielles avec les colonnes valides
final_columns = essential_columns + valid_columns

# Créer le DataFrame final avec les colonnes filtrées
filtered_df = final_df[final_columns]

# Afficher le DataFrame filtré
filtered_df

,Code département,Code circonscription législative,Nuance élu,% Voix/exprimés EXG,% Voix/inscrits RN,% Voix/exprimés RN,% Voix/inscrits UG,% Voix/exprimés UG,% Voix/inscrits ENS,% Voix/exprimés ENS,...,% Voix/inscrits UXD,% Voix/exprimés UXD,% Voix/inscrits EXD,% Voix/exprimés EXD,% Voix/inscrits UDI,% Voix/exprimés UDI,% Voix/inscrits HOR,% Voix/exprimés HOR,% Voix/inscrits SOC,% Voix/exprimés SOC
0,1,101,None,0.69,27.43,39.37,16.34,23.45,8.13,11.68,...,None,None,None,None,None,None,None,None,None,None
1,1,102,None,1.02,27.67,39.2,16.67,23.61,17.09,24.21,...,None,None,1.83,2.59,None,None,None,None,None,None
2,1,103,None,0.85,20.51,32.11,16.04,25.12,20.71,32.43,...,None,None,None,None,None,None,None,None,None,None
3,1,104,None,1.07,31.44,46.01,13.64,19.96,14.95,21.87,...,None,None,None,None,None,None,None,None,None,None
4,1,105,None,1.18,None,None,16.10,24.34,7.75,11.71,...,25.88,39.12,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,ZZ07,None,None,3.46,7.79,14.46,32.58,16.76,37.78,...,None,None,None,None,None,None,None,None,None,None
573,ZZ,ZZ08,None,None,None,None,5.05,23.14,5.27,24.18,...,None,None,None,None,None,None,None,None,None,None
574,ZZ,ZZ09,None,None,None,None,14.19,51.57,4.32,15.7,...,None,None,None,None,None,None,None,None,None,None
575,ZZ,ZZ10,None,None,None,None,10.23,32.52,10.01,31.82,...,5.4,17.16,None,None,None,None,None,None,None,None


In [15]:
circo241 = filtered_df.copy()
circo241 = circo241.drop(columns=['Code département'])
circo241

,Code circonscription législative,Nuance élu,% Voix/exprimés EXG,% Voix/inscrits RN,% Voix/exprimés RN,% Voix/inscrits UG,% Voix/exprimés UG,% Voix/inscrits ENS,% Voix/exprimés ENS,% Voix/inscrits LR,...,% Voix/inscrits UXD,% Voix/exprimés UXD,% Voix/inscrits EXD,% Voix/exprimés EXD,% Voix/inscrits UDI,% Voix/exprimés UDI,% Voix/inscrits HOR,% Voix/exprimés HOR,% Voix/inscrits SOC,% Voix/exprimés SOC
0,101,None,0.69,27.43,39.37,16.34,23.45,8.13,11.68,16.69,...,None,None,None,None,None,None,None,None,None,None
1,102,None,1.02,27.67,39.2,16.67,23.61,17.09,24.21,6.61,...,None,None,1.83,2.59,None,None,None,None,None,None
2,103,None,0.85,20.51,32.11,16.04,25.12,20.71,32.43,4.35,...,None,None,None,None,None,None,None,None,None,None
3,104,None,1.07,31.44,46.01,13.64,19.96,14.95,21.87,7.47,...,None,None,None,None,None,None,None,None,None,None
4,105,None,1.18,None,None,16.10,24.34,7.75,11.71,2.08,...,25.88,39.12,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ07,None,None,3.46,7.79,14.46,32.58,16.76,37.78,2.07,...,None,None,None,None,None,None,None,None,None,None
573,ZZ08,None,None,None,None,5.05,23.14,5.27,24.18,7.76,...,None,None,None,None,None,None,None,None,None,None
574,ZZ09,None,None,None,None,14.19,51.57,4.32,15.7,0.95,...,None,None,None,None,None,None,None,None,None,None
575,ZZ10,None,None,None,None,10.23,32.52,10.01,31.82,3.06,...,5.4,17.16,None,None,None,None,None,None,None,None


In [16]:
# Convertir la colonne de jointure en type str pour circo241
circo241['Code circonscription législative'] = circo241['Code circonscription législative'].astype(str)

# Convertir la colonne de jointure en type str pour circo242
circo242['Code circonscription législative'] = circo242['Code circonscription législative'].astype(str)

# Extraire les 4 premiers caractères, ajouter des zéros devant si nécessaire
circo241['Code circonscription législative'] = circo241['Code circonscription législative'].str[:5].str.zfill(5)
circo242['Code circonscription législative'] = circo242['Code circonscription législative'].str[:5].str.zfill(5)

In [17]:
# Ajouter le suffixe aux colonnes de circo242, sauf pour 'Code circonscription législative'
circo242.columns = [col + ' Legi_T224' if col != 'Code circonscription législative' else col for col in circo242.columns]

# Ajouter le suffixe aux colonnes de circo241, sauf pour 'Code circonscription législative'
circo241.columns = [col + ' Legi_T124' if col != 'Code circonscription législative' else col for col in circo241.columns]

# Fusionner les deux DataFrames sur 'Code circonscription législative'
merged_df = pd.merge(circo241, circo242, on='Code circonscription législative', how='left')
merged_df

,Code circonscription législative,Nuance élu Legi_T124,% Voix/exprimés EXG Legi_T124,% Voix/inscrits RN Legi_T124,% Voix/exprimés RN Legi_T124,% Voix/inscrits UG Legi_T124,% Voix/exprimés UG Legi_T124,% Voix/inscrits ENS Legi_T124,% Voix/exprimés ENS Legi_T124,% Voix/inscrits LR Legi_T124,...,% Voix/inscrits DIV Legi_T224,% Voix/exprimés DIV Legi_T224,% Voix/inscrits DVD Legi_T224,% Voix/exprimés DVD Legi_T224,% Voix/inscrits XD Legi_T224,% Voix/exprimés XD Legi_T224,% Voix/inscrits ENS Legi_T224,% Voix/exprimés ENS Legi_T224,% Voix/inscrits UG Legi_T224,% Voix/exprimés UG Legi_T224
0,00101,None,0.69,27.43,39.37,16.34,23.45,8.13,11.68,16.69,...,None,None,39.02,56.48,30.07,43.52,NaN,NaN,None,None
1,00102,None,1.02,27.67,39.2,16.67,23.61,17.09,24.21,6.61,...,None,None,None,None,31.17,44.91,38.24,55.09,None,None
2,00103,None,0.85,20.51,32.11,16.04,25.12,20.71,32.43,4.35,...,None,None,None,None,23.0,37.0,39.16,63.00,None,None
3,00104,None,1.07,31.44,46.01,13.64,19.96,14.95,21.87,7.47,...,None,None,None,None,34.52,51.35,32.70,48.65,None,None
4,00105,None,1.18,None,None,16.10,24.34,7.75,11.71,2.08,...,None,None,None,None,34.7,57.53,NaN,NaN,25.62,42.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0ZZ07,None,None,3.46,7.79,14.46,32.58,16.76,37.78,2.07,...,None,None,None,None,None,None,26.39,58.21,18.95,41.79
573,0ZZ08,None,None,None,None,5.05,23.14,5.27,24.18,7.76,...,None,None,11.03,47.3,None,None,12.29,52.70,None,None
574,0ZZ09,None,None,None,None,14.19,51.57,4.32,15.7,0.95,...,None,None,None,None,None,None,6.82,25.29,20.15,74.71
575,0ZZ10,None,None,None,None,10.23,32.52,10.01,31.82,3.06,...,None,None,None,None,None,None,14.98,53.23,13.16,46.77


In [18]:
merged_df.columns

Index(['Code circonscription législative', 'Nuance élu Legi_T124',
       '% Voix/exprimés EXG Legi_T124', '% Voix/inscrits RN Legi_T124',
       '% Voix/exprimés RN Legi_T124', '% Voix/inscrits UG Legi_T124',
       '% Voix/exprimés UG Legi_T124', '% Voix/inscrits ENS Legi_T124',
       '% Voix/exprimés ENS Legi_T124', '% Voix/inscrits LR Legi_T124',
       '% Voix/exprimés LR Legi_T124', '% Voix/exprimés REC Legi_T124',
       '% Voix/inscrits DIV Legi_T124', '% Voix/exprimés DIV Legi_T124',
       '% Voix/inscrits DVD Legi_T124', '% Voix/exprimés DVD Legi_T124',
       '% Voix/inscrits DVC Legi_T124', '% Voix/exprimés DVC Legi_T124',
       '% Voix/inscrits ECO Legi_T124', '% Voix/exprimés ECO Legi_T124',
       '% Voix/inscrits DVG Legi_T124', '% Voix/exprimés DVG Legi_T124',
       '% Voix/inscrits DSV Legi_T124', '% Voix/exprimés DSV Legi_T124',
       '% Voix/inscrits REG Legi_T124', '% Voix/exprimés REG Legi_T124',
       '% Voix/inscrits UXD Legi_T124', '% Voix/exprimés UXD Le

In [19]:
# Fonction pour combiner les nuances et signaler les valeurs non-None
def combine_nuances(row):
    nuance_124 = row.get('Nuance élu Legi_T124', None)
    nuance_224 = row.get('Nuance élu Legi_T224', None)
    
    if pd.notna(nuance_124) and pd.notna(nuance_224):
        print(f"Code circonscription législative {row['Code circonscription législative']}: pas none")
        if nuance_124 != nuance_224:
            # Si les deux sont non-None mais différentes, afficher un message d'erreur
            raise ValueError(f"Conflit dans la circonscription législative {row['Code circonscription législative']}: '{nuance_124}' vs '{nuance_224}'")
        return nuance_124
    elif pd.notna(nuance_124):
        print(f"Code circonscription législative {row['Code circonscription législative']}: Legi_T224 est None")
        return nuance_124
    elif pd.notna(nuance_224):
        print(f"Code circonscription législative {row['Code circonscription législative']}: Legi_T124 est None")
        return nuance_224
    else:
        print(f"Code circonscription législative {row['Code circonscription législative']}: both none")
        return None

# Appliquer la fonction pour créer la nouvelle colonne
merged_df['Nuance élu Legi24'] = merged_df.apply(combine_nuances, axis=1)
merged_df

Code circonscription législative 00101: Legi_T124 est None
Code circonscription législative 00102: Legi_T124 est None
Code circonscription législative 00103: Legi_T124 est None
Code circonscription législative 00104: Legi_T124 est None
Code circonscription législative 00105: Legi_T124 est None
Code circonscription législative 00201: Legi_T224 est None
Code circonscription législative 00202: Legi_T124 est None
Code circonscription législative 00203: Legi_T224 est None
Code circonscription législative 00204: Legi_T224 est None
Code circonscription législative 00205: Legi_T224 est None
Code circonscription législative 00301: Legi_T124 est None
Code circonscription législative 00302: Legi_T124 est None
Code circonscription législative 00303: Legi_T124 est None
Code circonscription législative 00401: Legi_T124 est None
Code circonscription législative 00402: Legi_T124 est None
Code circonscription législative 00501: Legi_T124 est None
Code circonscription législative 00502: Legi_T124 est No

,Code circonscription législative,Nuance élu Legi_T124,% Voix/exprimés EXG Legi_T124,% Voix/inscrits RN Legi_T124,% Voix/exprimés RN Legi_T124,% Voix/inscrits UG Legi_T124,% Voix/exprimés UG Legi_T124,% Voix/inscrits ENS Legi_T124,% Voix/exprimés ENS Legi_T124,% Voix/inscrits LR Legi_T124,...,% Voix/exprimés DIV Legi_T224,% Voix/inscrits DVD Legi_T224,% Voix/exprimés DVD Legi_T224,% Voix/inscrits XD Legi_T224,% Voix/exprimés XD Legi_T224,% Voix/inscrits ENS Legi_T224,% Voix/exprimés ENS Legi_T224,% Voix/inscrits UG Legi_T224,% Voix/exprimés UG Legi_T224,Nuance élu Legi24
0,00101,None,0.69,27.43,39.37,16.34,23.45,8.13,11.68,16.69,...,None,39.02,56.48,30.07,43.52,NaN,NaN,None,None,DVD
1,00102,None,1.02,27.67,39.2,16.67,23.61,17.09,24.21,6.61,...,None,None,None,31.17,44.91,38.24,55.09,None,None,ENS
2,00103,None,0.85,20.51,32.11,16.04,25.12,20.71,32.43,4.35,...,None,None,None,23.0,37.0,39.16,63.00,None,None,ENS
3,00104,None,1.07,31.44,46.01,13.64,19.96,14.95,21.87,7.47,...,None,None,None,34.52,51.35,32.70,48.65,None,None,XD
4,00105,None,1.18,None,None,16.10,24.34,7.75,11.71,2.08,...,None,None,None,34.7,57.53,NaN,NaN,25.62,42.47,XD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0ZZ07,None,None,3.46,7.79,14.46,32.58,16.76,37.78,2.07,...,None,None,None,None,None,26.39,58.21,18.95,41.79,ENS
573,0ZZ08,None,None,None,None,5.05,23.14,5.27,24.18,7.76,...,None,11.03,47.3,None,None,12.29,52.70,None,None,ENS
574,0ZZ09,None,None,None,None,14.19,51.57,4.32,15.7,0.95,...,None,None,None,None,None,6.82,25.29,20.15,74.71,UG
575,0ZZ10,None,None,None,None,10.23,32.52,10.01,31.82,3.06,...,None,None,None,None,None,14.98,53.23,13.16,46.77,ENS


In [20]:
print("\nPremière ligne du DataFrame merged_df :")
print(merged_df.iloc[0])


Première ligne du DataFrame merged_df :
Code circonscription législative     00101
Nuance élu Legi_T124                  None
% Voix/exprimés EXG Legi_T124         0.69
% Voix/inscrits RN Legi_T124         27.43
% Voix/exprimés RN Legi_T124         39.37
% Voix/inscrits UG Legi_T124         16.34
% Voix/exprimés UG Legi_T124         23.45
% Voix/inscrits ENS Legi_T124         8.13
% Voix/exprimés ENS Legi_T124        11.68
% Voix/inscrits LR Legi_T124         16.69
% Voix/exprimés LR Legi_T124         23.96
% Voix/exprimés REC Legi_T124         None
% Voix/inscrits DIV Legi_T124         None
% Voix/exprimés DIV Legi_T124         None
% Voix/inscrits DVD Legi_T124         None
% Voix/exprimés DVD Legi_T124         None
% Voix/inscrits DVC Legi_T124         None
% Voix/exprimés DVC Legi_T124         None
% Voix/inscrits ECO Legi_T124         None
% Voix/exprimés ECO Legi_T124         None
% Voix/inscrits DVG Legi_T124         None
% Voix/exprimés DVG Legi_T124         None
% Voix/inscri

In [21]:
merged_df = merged_df.drop(['Nuance élu Legi_T224', 'Nuance élu Legi_T124'], axis=1)
merged_df.columns

Index(['Code circonscription législative', '% Voix/exprimés EXG Legi_T124',
       '% Voix/inscrits RN Legi_T124', '% Voix/exprimés RN Legi_T124',
       '% Voix/inscrits UG Legi_T124', '% Voix/exprimés UG Legi_T124',
       '% Voix/inscrits ENS Legi_T124', '% Voix/exprimés ENS Legi_T124',
       '% Voix/inscrits LR Legi_T124', '% Voix/exprimés LR Legi_T124',
       '% Voix/exprimés REC Legi_T124', '% Voix/inscrits DIV Legi_T124',
       '% Voix/exprimés DIV Legi_T124', '% Voix/inscrits DVD Legi_T124',
       '% Voix/exprimés DVD Legi_T124', '% Voix/inscrits DVC Legi_T124',
       '% Voix/exprimés DVC Legi_T124', '% Voix/inscrits ECO Legi_T124',
       '% Voix/exprimés ECO Legi_T124', '% Voix/inscrits DVG Legi_T124',
       '% Voix/exprimés DVG Legi_T124', '% Voix/inscrits DSV Legi_T124',
       '% Voix/exprimés DSV Legi_T124', '% Voix/inscrits REG Legi_T124',
       '% Voix/exprimés REG Legi_T124', '% Voix/inscrits UXD Legi_T124',
       '% Voix/exprimés UXD Legi_T124', '% Voix/inscri

In [23]:
merged_df.to_excel('Cleaned_Data/clean_Legi24.xlsx', index=False)

# Législatives 2022 #

In [466]:
# Obtenez le chemin du répertoire actuel (qui est RisqueRNAnalyse)
current_dir = os.getcwd()

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(current_dir, 'Database', 'Legislatives2022CircoT2.xlsx')

# Chargez le fichier Excel dans un DataFrame
Legi2022CircoT2 = pd.read_excel(fichier_excel)

In [467]:
Legi2022CircoT2.columns

Index(['Code du département', 'Libellé du département',
       'Code de la circonscription', 'Libellé de la circonscription',
       'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
       '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls',
       '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot',
       'N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Nuance', 'Voix', '% Voix/Ins',
       '% Voix/Exp', 'Sièges', 'N°Panneau 2', 'Sexe 2', 'Nom 2', 'Prénom 2',
       'Nuance 2', 'Voix 2', '% Voix/Ins 2', '% Voix/Exp 2', 'Sièges 2',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Nuance 3',
       'Unnamed: 42', '% Voix/Ins 3', '% Voix/Exp 3', 'Siège 3'],
      dtype='object')

In [468]:
circo222 = Legi2022CircoT2[['Code du département', 'Code de la circonscription', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
       '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls',
       '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Nuance', 'Voix', '% Voix/Ins',
       '% Voix/Exp', 'Sièges', 'Nuance 2', 'Voix 2', '% Voix/Ins 2', '% Voix/Exp 2', 'Sièges 2', '% Voix/Ins 3', '% Voix/Exp 3', 'Siège 3', 'Nuance 3',]]
circo222

,Code du département,Code de la circonscription,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,...,Sièges,Nuance 2,Voix 2,% Voix/Ins 2,% Voix/Exp 2,Sièges 2,% Voix/Ins 3,% Voix/Exp 3,Siège 3,Nuance 3
0,01,1,86194,45446,52.73,40748,47.27,1587,1.84,3.89,...,NaN,LR,24407.0,28.32,63.22,Elu,NaN,NaN,NaN,NaN
1,01,2,99979,53307,53.32,46672,46.68,3033,3.03,6.50,...,NaN,ENS,24960.0,24.97,58.34,Elu,NaN,NaN,NaN,NaN
2,01,3,82207,48291,58.74,33916,41.26,1870,2.27,5.51,...,NaN,ENS,18398.0,22.38,58.72,Elu,NaN,NaN,NaN,NaN
3,01,4,94903,51702,54.48,43201,45.52,5450,5.74,12.62,...,NaN,RN,22601.0,23.81,62.27,Elu,NaN,NaN,NaN,NaN
4,01,5,77466,43704,56.42,33762,43.58,2354,3.04,6.97,...,NaN,DVD,17687.0,22.83,57.86,Elu,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ZZ,7,123403,83482,67.65,39921,32.35,1317,1.07,3.30,...,NaN,ENS,23191.0,18.79,60.21,Elu,NaN,NaN,NaN,NaN
568,ZZ,8,131216,112933,86.07,18283,13.93,1468,1.12,8.03,...,NaN,UDI,8470.0,6.46,50.58,Elu,NaN,NaN,NaN,NaN
569,ZZ,9,120553,98709,81.88,21844,18.12,795,0.66,3.64,...,Elu,ENS,9639.0,8.00,45.93,NaN,NaN,NaN,NaN,NaN
570,ZZ,10,104452,82695,79.17,21757,20.83,1136,1.09,5.22,...,NaN,ENS,13048.0,12.49,63.58,Elu,NaN,NaN,NaN,NaN


In [469]:
# Remplacer les valeurs
circo222['Nuance'] = circo222['Nuance'].replace({'UDI': 'LR', 'DVD': 'LR'})
# Remplacer les valeurs
circo222['Nuance 2'] = circo222['Nuance 2'].replace({'UDI': 'LR', 'DVD': 'LR', 'DSV' : 'RN'})

/tmp/ipykernel_292/1513533403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circo222['Nuance'] = circo222['Nuance'].replace({'UDI': 'LR', 'DVD': 'LR'})
/tmp/ipykernel_292/1513533403.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circo222['Nuance 2'] = circo222['Nuance 2'].replace({'UDI': 'LR', 'DVD': 'LR', 'DSV' : 'RN'})


In [470]:
# Liste des colonnes de nuances et des colonnes d'élu
nuance_columns = ['Nuance', 'Nuance 2', 'Nuance 3']
elu_columns = ['Sièges', 'Sièges 2', 'Siège 3']

# Fonction pour obtenir la nuance du candidat élu
def get_elected_nuance(row):
    for nuance_col, elu_col in zip(nuance_columns, elu_columns):
        if row[elu_col] == 'Elu':
            return row[nuance_col]
    return None

circo222['Nuance élu'] = circo222.apply(get_elected_nuance, axis=1)
circo222

/tmp/ipykernel_292/388863087.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circo222['Nuance élu'] = circo222.apply(get_elected_nuance, axis=1)


,Code du département,Code de la circonscription,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,...,Nuance 2,Voix 2,% Voix/Ins 2,% Voix/Exp 2,Sièges 2,% Voix/Ins 3,% Voix/Exp 3,Siège 3,Nuance 3,Nuance élu
0,01,1,86194,45446,52.73,40748,47.27,1587,1.84,3.89,...,LR,24407.0,28.32,63.22,Elu,NaN,NaN,NaN,NaN,LR
1,01,2,99979,53307,53.32,46672,46.68,3033,3.03,6.50,...,ENS,24960.0,24.97,58.34,Elu,NaN,NaN,NaN,NaN,ENS
2,01,3,82207,48291,58.74,33916,41.26,1870,2.27,5.51,...,ENS,18398.0,22.38,58.72,Elu,NaN,NaN,NaN,NaN,ENS
3,01,4,94903,51702,54.48,43201,45.52,5450,5.74,12.62,...,RN,22601.0,23.81,62.27,Elu,NaN,NaN,NaN,NaN,RN
4,01,5,77466,43704,56.42,33762,43.58,2354,3.04,6.97,...,LR,17687.0,22.83,57.86,Elu,NaN,NaN,NaN,NaN,LR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ZZ,7,123403,83482,67.65,39921,32.35,1317,1.07,3.30,...,ENS,23191.0,18.79,60.21,Elu,NaN,NaN,NaN,NaN,ENS
568,ZZ,8,131216,112933,86.07,18283,13.93,1468,1.12,8.03,...,LR,8470.0,6.46,50.58,Elu,NaN,NaN,NaN,NaN,LR
569,ZZ,9,120553,98709,81.88,21844,18.12,795,0.66,3.64,...,ENS,9639.0,8.00,45.93,NaN,NaN,NaN,NaN,NaN,NUP
570,ZZ,10,104452,82695,79.17,21757,20.83,1136,1.09,5.22,...,ENS,13048.0,12.49,63.58,Elu,NaN,NaN,NaN,NaN,ENS


In [471]:
circo222 =  circo222.rename(columns={'Code du département' : 'Code département', 'Code de la circonscription' : 'Code de la circonscription législative'})

In [472]:
# Convertir les colonnes en chaînes de caractères
circo222['Code du département'] = circo222['Code département'].astype(str)
circo222['Code de la circonscription'] = circo222['Code de la circonscription législative'].astype(str)

# Combiner les deux colonnes
circo222['Code circo'] = circo222['Code du département'] + circo222['Code de la circonscription']

# Compléter à longueur 5 avec des zéros devant
circo222['Code circo'] = circo222['Code circo'].str.zfill(5)

In [473]:
circo222.columns

Index(['Code département', 'Code de la circonscription législative',
       'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins',
       'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
       '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Nuance', 'Voix',
       '% Voix/Ins', '% Voix/Exp', 'Sièges', 'Nuance 2', 'Voix 2',
       '% Voix/Ins 2', '% Voix/Exp 2', 'Sièges 2', '% Voix/Ins 3',
       '% Voix/Exp 3', 'Siège 3', 'Nuance 3', 'Nuance élu',
       'Code du département', 'Code de la circonscription', 'Code circo'],
      dtype='object')

In [474]:
nuances_list = ['NUP', 'ENS', 'LR', 'REG', 'DVG', 'RN', 'DVC', 'DIV', 'DXG']

# Créer les nouvelles colonnes pour chaque nuance
for nuance in nuances_list:
    # Créer les noms des nouvelles colonnes
    ins_col = f'% Voix/Ins {nuance}'
    exp_col = f'% Voix/Exp {nuance}'

    # Calculer les valeurs pour '% Voix/Ins {nuance}' en fonction de Nuance, Nuance 2, et Nuance 3
    circo222[ins_col] = circo222.apply(
        lambda row: row['% Voix/Ins'] if row['Nuance'] == nuance 
                    else (row['% Voix/Ins 2'] if row['Nuance 2'] == nuance  
                          else row['% Voix/Ins 3'] if row['Nuance 3'] == nuance  
                              else None),
        axis=1
    )

    # Calculer les valeurs pour '% Voix/Exp {nuance}' en fonction de Nuance, Nuance 2, et Nuance 3
    circo222[exp_col] = circo222.apply(
        lambda row: row['% Voix/Exp'] if row['Nuance'] == nuance 
                    else (row['% Voix/Exp 2'] if row['Nuance 2'] == nuance
                          else row['% Voix/Exp 3'] if row['Nuance 3'] == nuance 
                              else None),
        axis=1
    )

            
circo222

,Code département,Code de la circonscription législative,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,...,% Voix/Ins DVG,% Voix/Exp DVG,% Voix/Ins RN,% Voix/Exp RN,% Voix/Ins DVC,% Voix/Exp DVC,% Voix/Ins DIV,% Voix/Exp DIV,% Voix/Ins DXG,% Voix/Exp DXG
0,01,1,86194,45446,52.73,40748,47.27,1587,1.84,3.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2,99979,53307,53.32,46672,46.68,3033,3.03,6.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,3,82207,48291,58.74,33916,41.26,1870,2.27,5.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,4,94903,51702,54.48,43201,45.52,5450,5.74,12.62,...,NaN,NaN,23.81,62.27,NaN,NaN,NaN,NaN,NaN,NaN
4,01,5,77466,43704,56.42,33762,43.58,2354,3.04,6.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ZZ,7,123403,83482,67.65,39921,32.35,1317,1.07,3.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568,ZZ,8,131216,112933,86.07,18283,13.93,1468,1.12,8.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569,ZZ,9,120553,98709,81.88,21844,18.12,795,0.66,3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,ZZ,10,104452,82695,79.17,21757,20.83,1136,1.09,5.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [475]:
circo222.columns

Index(['Code département', 'Code de la circonscription législative',
       'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins',
       'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins',
       '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Nuance', 'Voix',
       '% Voix/Ins', '% Voix/Exp', 'Sièges', 'Nuance 2', 'Voix 2',
       '% Voix/Ins 2', '% Voix/Exp 2', 'Sièges 2', '% Voix/Ins 3',
       '% Voix/Exp 3', 'Siège 3', 'Nuance 3', 'Nuance élu',
       'Code du département', 'Code de la circonscription', 'Code circo',
       '% Voix/Ins NUP', '% Voix/Exp NUP', '% Voix/Ins ENS', '% Voix/Exp ENS',
       '% Voix/Ins LR', '% Voix/Exp LR', '% Voix/Ins REG', '% Voix/Exp REG',
       '% Voix/Ins DVG', '% Voix/Exp DVG', '% Voix/Ins RN', '% Voix/Exp RN',
       '% Voix/Ins DVC', '% Voix/Exp DVC', '% Voix/Ins DIV', '% Voix/Exp DIV',
       '% Voix/Ins DXG', '% Voix/Exp DXG'],
      dtype='object')

In [476]:
circo222 = circo222[[ 'Code circo',
       '% Voix/Ins NUP', '% Voix/Exp NUP', '% Voix/Ins ENS', '% Voix/Exp ENS',
       '% Voix/Ins LR', '% Voix/Exp LR', '% Voix/Ins REG', '% Voix/Exp REG',
       '% Voix/Ins DVG', '% Voix/Exp DVG', '% Voix/Ins RN', '% Voix/Exp RN',
       '% Voix/Ins DVC', '% Voix/Exp DVC', '% Voix/Ins DIV', '% Voix/Exp DIV',
       '% Voix/Ins DXG', '% Voix/Exp DXG', 'Nuance élu']]
circo222

,Code circo,% Voix/Ins NUP,% Voix/Exp NUP,% Voix/Ins ENS,% Voix/Exp ENS,% Voix/Ins LR,% Voix/Exp LR,% Voix/Ins REG,% Voix/Exp REG,% Voix/Ins DVG,% Voix/Exp DVG,% Voix/Ins RN,% Voix/Exp RN,% Voix/Ins DVC,% Voix/Exp DVC,% Voix/Ins DIV,% Voix/Exp DIV,% Voix/Ins DXG,% Voix/Exp DXG,Nuance élu
0,00011,16.48,36.78,NaN,NaN,28.32,63.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
1,00012,17.83,41.66,24.97,58.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
2,00013,15.73,41.28,22.38,58.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
3,00014,14.43,37.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.81,62.27,NaN,NaN,NaN,NaN,NaN,NaN,RN
4,00015,16.63,42.14,NaN,NaN,22.83,57.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,00ZZ7,12.42,39.79,18.79,60.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
568,00ZZ8,NaN,NaN,6.31,49.42,6.46,50.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
569,00ZZ9,9.41,54.07,8.00,45.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NUP
570,0ZZ10,7.16,36.42,12.49,63.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS


In [477]:
#Ajout victoires premiers tours 2024
# Obtenez le chemin du répertoire actuel (qui est RisqueRNAnalyse)
current_dir = os.getcwd()

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(current_dir, 'Database', 'Legislatives2022CircoT1.xlsx')

# Chargez le fichier Excel dans un DataFrame
Legi2022CircoT1 = pd.read_excel(fichier_excel)
Legi2022CircoT1

,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Siège 21,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Nuance 22,Voix 22,% Voix/Ins 22,% Voix/Exp 22,Siège 22
0,01,Ain,1,1ère circonscription,Complet,86187,43652,50.65,42535,49.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Ain,2,2ème circonscription,Complet,99953,50270,50.29,49683,49.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Ain,3,3ème circonscription,Complet,82204,45945,55.89,36259,44.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,Ain,4,4ème circonscription,Complet,94897,49258,51.91,45639,48.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,Ain,5,5ème circonscription,Complet,77456,40382,52.14,37074,47.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,Français établis hors de France,7,7ème circonscription,Complet,123409,87199,70.66,36210,29.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
573,ZZ,Français établis hors de France,8,8ème circonscription,Complet,131219,115176,87.77,16043,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,ZZ,Français établis hors de France,9,9ème circonscription,Complet,120556,102826,85.29,17730,14.71,...,NaN,21.0,M,MARTINEZ,Jean-Claude,DXD,101.0,0.08,0.58,NaN
575,ZZ,Français établis hors de France,10,10ème circonscription,Complet,104303,83943,80.48,20360,19.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [478]:
circo221 = Legi2022CircoT1.copy()
# Convertir les colonnes en chaînes de caractères
circo221['Code du département'] = circo221['Code du département'].astype(str)
circo221['Code de la circonscription'] = circo221['Code de la circonscription'].astype(str)

# Combiner les deux colonnes
circo221['Code circo'] = circo221['Code du département'] + circo221['Code de la circonscription']

# Compléter à longueur 5 avec des zéros devant
circo221['Code circo'] = circo221['Code circo'].str.zfill(5)

In [479]:
circo221

,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Nuance 22,Voix 22,% Voix/Ins 22,% Voix/Exp 22,Siège 22,Code circo
0,01,Ain,1,1ère circonscription,Complet,86187,43652,50.65,42535,49.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00011
1,01,Ain,2,2ème circonscription,Complet,99953,50270,50.29,49683,49.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00012
2,01,Ain,3,3ème circonscription,Complet,82204,45945,55.89,36259,44.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00013
3,01,Ain,4,4ème circonscription,Complet,94897,49258,51.91,45639,48.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00014
4,01,Ain,5,5ème circonscription,Complet,77456,40382,52.14,37074,47.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,Français établis hors de France,7,7ème circonscription,Complet,123409,87199,70.66,36210,29.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00ZZ7
573,ZZ,Français établis hors de France,8,8ème circonscription,Complet,131219,115176,87.77,16043,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00ZZ8
574,ZZ,Français établis hors de France,9,9ème circonscription,Complet,120556,102826,85.29,17730,14.71,...,21.0,M,MARTINEZ,Jean-Claude,DXD,101.0,0.08,0.58,NaN,00ZZ9
575,ZZ,Français établis hors de France,10,10ème circonscription,Complet,104303,83943,80.48,20360,19.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0ZZ10


In [480]:
# Sélectionner les colonnes spécifiques
columns_to_extract = [
    'Code circo', '% Abs/Ins', '% Vot/Ins', '% Blancs/Ins', '% Blancs/Vot',
    '% Nuls/Ins', '% Nuls/Vot', '% Exp/Ins', '% Exp/Vot', 'Sièges',
    '% Voix/Ins', '% Voix/Exp', 'Siège 2', '% Voix/Ins 2', '% Voix/Exp 2',
    'Siège 3', '% Voix/Ins 3', '% Voix/Exp 3', 'Siège 4', '% Voix/Ins 4',
    '% Voix/Exp 4', 'Siège 5', '% Voix/Ins 5', '% Voix/Exp 5', 'Siège 6',
    '% Voix/Ins 6', '% Voix/Exp 6', 'Siège 7', '% Voix/Ins 7', '% Voix/Exp 7',
    'Siège 8', '% Voix/Ins 8', '% Voix/Exp 8', 'Siège 9', '% Voix/Ins 9',
    '% Voix/Exp 8.1', 'Siège 10', '% Voix/Ins 10', '% Voix/Exp 10',
    'Siège 11', '% Voix/Ins 11', '% Voix/Exp 11', 'Siège 12', '% Voix/Ins 12',
    '% Voix/Exp 12', 'Siège 13', '% Voix/Ins 13', '% Voix/Exp 13', 'Siège 14',
    '% Voix/Ins 14', '% Voix/Exp 14', 'Siège 15', '% Voix/Ins 15', '% Voix/Exp 15',
    'Siège 16', '% Voix/Ins 16', '% Voix/Exp 16', 'Siège 17', '% Voix/Ins 17',
    '% Voix/Exp 17', 'Siège 18', '% Voix/Ins 18', '% Voix/Exp 18', 'Siège 19',
    '% Voix/Ins 19', '% Voix/Exp 19', 'Siège 20', '% Voix/Ins 20', '% Voix/Exp 20',
    'Siège 21', '% Voix/Ins 21', '% Voix/Exp 21', 'Siège 22', '% Voix/Ins 22',
    '% Voix/Exp 22', 'Nuance',  'Nuance 2',  'Nuance 3', 'Nuance 4', 'Nuance 5', 'Nuance 6', 'Nuance 7', 'Nuance 8',  'Nuance 9', 'Nuance 10',  'Nuance 11', 'Nuance 12', 
    'Nuance 13',  'Nuance 14', 'Nuance 15',  'Nuance 16', 'Nuance 17', 'Nuance 18',  'Nuance 19', 'Nuance 20', 'Nuance 21', 'Nuance 22'
]

# Sélectionner les colonnes du DataFrame circo221
extract221 = circo221[columns_to_extract]

# Renommer la colonne '% Voix/Exp 8.1' en '% Voix/Exp 9'
extract221.rename(columns={'% Voix/Exp 8.1': '% Voix/Exp 9'}, inplace=True)
extract221

/tmp/ipykernel_292/2530383469.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract221.rename(columns={'% Voix/Exp 8.1': '% Voix/Exp 9'}, inplace=True)


,Code circo,% Abs/Ins,% Vot/Ins,% Blancs/Ins,% Blancs/Vot,% Nuls/Ins,% Nuls/Vot,% Exp/Ins,% Exp/Vot,Sièges,...,Nuance 13,Nuance 14,Nuance 15,Nuance 16,Nuance 17,Nuance 18,Nuance 19,Nuance 20,Nuance 21,Nuance 22
0,00011,50.65,49.35,0.57,1.15,0.27,0.55,48.51,98.30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00012,50.29,49.71,0.63,1.27,0.17,0.34,48.91,98.40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00013,55.89,44.11,0.47,1.06,0.18,0.40,43.46,98.54,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00014,51.91,48.09,0.67,1.40,0.22,0.46,47.20,98.14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00015,52.14,47.86,1.06,2.21,0.27,0.56,46.54,97.23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,00ZZ7,70.66,29.34,0.22,0.74,0.04,0.15,29.08,99.11,NaN,...,DVD,DVD,REC,RN,NaN,NaN,NaN,NaN,NaN,NaN
573,00ZZ8,87.77,12.23,0.14,1.12,0.01,0.10,12.08,98.78,NaN,...,RN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,00ZZ9,85.29,14.71,0.33,2.21,0.06,0.39,14.32,97.39,NaN,...,DVC,DVC,DVC,DVC,DVC,LR,DVD,REC,RN,DXD
575,0ZZ10,80.48,19.52,0.28,1.42,0.04,0.22,19.20,98.35,NaN,...,DVD,DSV,REC,RN,NaN,NaN,NaN,NaN,NaN,NaN


In [481]:
# Définir les colonnes de nuance et d'élu
nuance_columns = ['Nuance'] + [f'Nuance {i}' for i in range(2, 16)]
elu_columns = ['Sièges'] + [f'Siège {i}' for i in range(2, 23)]

# Fonction pour obtenir la nuance du candidat élu
def get_elected_nuance(row):
    for nuance_col, elu_col in zip(nuance_columns, elu_columns):
        if row[elu_col] == 'Elu':
            return row[nuance_col]
    return None

# Créer la colonne 'Nuance élu'
extract221['Nuance élu'] = extract221.apply(get_elected_nuance, axis=1)
extract221['Nuance élu'].value_counts()

/tmp/ipykernel_292/2991656268.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract221['Nuance élu'] = extract221.apply(get_elected_nuance, axis=1)


Nuance élu
NUP    4
ENS    1
Name: count, dtype: int64

In [482]:
# Étape 1 : Remplacer les valeurs spécifiques
# Attribution de RN à DSV et DXD
extract221['Nuance'] = extract221['Nuance'].replace({'RN': 'DSV'})
extract221['Nuance 2'] = extract221['Nuance 2'].replace({'RN': 'DSV'})
extract221['Nuance'] = extract221['Nuance'].replace({'RN': 'DXD'})
extract221['Nuance 2'] = extract221['Nuance 2'].replace({'RN': 'DXD'})
extract221['Nuance'] = extract221['Nuance'].replace({'RN': 'REC'})
extract221['Nuance 2'] = extract221['Nuance 2'].replace({'RN': 'REC'})

# Attribution de KR à UDI et DVD
extract221['Nuance'] = extract221['Nuance'].replace({'LR': 'UDI'})
extract221['Nuance 2'] = extract221['Nuance 2'].replace({'LR': 'UDI'})
extract221['Nuance'] = extract221['Nuance'].replace({'LR': 'DVD'})
extract221['Nuance 2'] = extract221['Nuance 2'].replace({'LR': 'DVD'})

nuances_list = ['DIV', 'DVC', 'DVG', 'DXG', 'ECO', 'ENS', 'LR', 'NUP', 'RDG', 'REG', 'RN']

# Initialiser un DataFrame pour les données réorganisées
reorganized_columns = [ 'Code circo', 'Nuance élu'] + [f'% Voix/Ins {nuance}' for nuance in nuances_list] + [f'% Voix/Exp {nuance}' for nuance in nuances_list]
reorganized_df = pd.DataFrame(columns=reorganized_columns)

# Copier les colonnes de base
reorganized_df['Code circo'] = extract221['Code circo']
reorganized_df['Nuance élu'] = extract221['Nuance élu']

# Créer des colonnes pour chaque nuance et score
for nuance in nuances_list:
    # Trouver les colonnes de score associées à chaque nuance
    score_ins_cols = [f'% Voix/Ins'] + [f'% Voix/Ins {i}' for i in range(2, 23)]
    score_exp_cols = [f'% Voix/Exp'] + [f'% Voix/Exp {i}' for i in range(2, 23)]
    nuance_cols = [f'Nuance'] + [f'Nuance {i}' for i in range(2, 23)]

    # Assigner les scores pour '% Voix/Ins'
    reorganized_df[f'% Voix/Ins {nuance}'] = extract221.apply(
        lambda row: next((row[score] for score, nuance_col in zip(score_ins_cols, nuance_cols) 
                          if row[nuance_col] == nuance and pd.notna(row[score])), None), axis=1
    )

    # Assigner les scores pour '% Voix/Exp'
    reorganized_df[f'% Voix/Exp {nuance}'] = extract221.apply(
        lambda row: next((row[score] for score, nuance_col in zip(score_exp_cols, nuance_cols) 
                          if row[nuance_col] == nuance and pd.notna(row[score])), None), axis=1
    )

reorganized_df

/tmp/ipykernel_292/2279150248.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract221['Nuance'] = extract221['Nuance'].replace({'RN': 'DSV'})
/tmp/ipykernel_292/2279150248.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract221['Nuance 2'] = extract221['Nuance 2'].replace({'RN': 'DSV'})
/tmp/ipykernel_292/2279150248.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,Code circo,Nuance élu,% Voix/Ins DIV,% Voix/Ins DVC,% Voix/Ins DVG,% Voix/Ins DXG,% Voix/Ins ECO,% Voix/Ins ENS,% Voix/Ins LR,% Voix/Ins NUP,...,% Voix/Exp DVC,% Voix/Exp DVG,% Voix/Exp DXG,% Voix/Exp ECO,% Voix/Exp ENS,% Voix/Exp LR,% Voix/Exp NUP,% Voix/Exp RDG,% Voix/Exp REG,% Voix/Exp RN
0,00011,None,NaN,NaN,NaN,0.45,1.35,9.36,12.30,11.58,...,NaN,NaN,0.94,2.78,19.30,25.35,23.87,NaN,NaN,21.46
1,00012,None,0.27,NaN,NaN,0.42,0.65,12.92,7.20,12.43,...,NaN,NaN,0.85,1.34,26.42,14.71,25.42,NaN,NaN,23.23
2,00013,None,0.00,0.08,NaN,0.34,2.14,13.02,7.82,9.72,...,0.19,NaN,0.79,4.93,29.96,17.98,22.36,NaN,NaN,13.97
3,00014,None,NaN,4.44,0.01,0.32,1.18,7.71,7.59,8.20,...,9.40,0.02,0.69,2.50,16.34,16.07,17.38,4.95,NaN,24.82
4,00015,None,0.91,NaN,0.36,0.53,NaN,NaN,4.62,10.95,...,NaN,0.78,1.15,NaN,NaN,9.93,23.54,NaN,NaN,19.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,00ZZ7,None,0.32,1.66,0.55,NaN,0.30,10.06,1.13,7.58,...,5.69,1.89,NaN,1.03,34.58,3.88,26.06,NaN,0.4,1.28
573,00ZZ8,None,0.43,0.16,0.05,NaN,0.17,3.35,NaN,2.28,...,1.29,0.41,NaN,1.43,27.78,NaN,18.84,0.33,NaN,1.87
574,00ZZ9,None,0.01,0.09,0.14,NaN,NaN,4.02,0.66,5.73,...,0.63,0.96,NaN,NaN,28.06,4.62,39.99,NaN,NaN,1.49
575,0ZZ10,None,0.02,1.04,0.23,NaN,0.26,6.29,2.08,4.33,...,5.41,1.18,NaN,1.38,32.75,10.85,22.54,NaN,NaN,2.10


In [483]:
circo221 = reorganized_df.copy()
circo221

,Code circo,Nuance élu,% Voix/Ins DIV,% Voix/Ins DVC,% Voix/Ins DVG,% Voix/Ins DXG,% Voix/Ins ECO,% Voix/Ins ENS,% Voix/Ins LR,% Voix/Ins NUP,...,% Voix/Exp DVC,% Voix/Exp DVG,% Voix/Exp DXG,% Voix/Exp ECO,% Voix/Exp ENS,% Voix/Exp LR,% Voix/Exp NUP,% Voix/Exp RDG,% Voix/Exp REG,% Voix/Exp RN
0,00011,None,NaN,NaN,NaN,0.45,1.35,9.36,12.30,11.58,...,NaN,NaN,0.94,2.78,19.30,25.35,23.87,NaN,NaN,21.46
1,00012,None,0.27,NaN,NaN,0.42,0.65,12.92,7.20,12.43,...,NaN,NaN,0.85,1.34,26.42,14.71,25.42,NaN,NaN,23.23
2,00013,None,0.00,0.08,NaN,0.34,2.14,13.02,7.82,9.72,...,0.19,NaN,0.79,4.93,29.96,17.98,22.36,NaN,NaN,13.97
3,00014,None,NaN,4.44,0.01,0.32,1.18,7.71,7.59,8.20,...,9.40,0.02,0.69,2.50,16.34,16.07,17.38,4.95,NaN,24.82
4,00015,None,0.91,NaN,0.36,0.53,NaN,NaN,4.62,10.95,...,NaN,0.78,1.15,NaN,NaN,9.93,23.54,NaN,NaN,19.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,00ZZ7,None,0.32,1.66,0.55,NaN,0.30,10.06,1.13,7.58,...,5.69,1.89,NaN,1.03,34.58,3.88,26.06,NaN,0.4,1.28
573,00ZZ8,None,0.43,0.16,0.05,NaN,0.17,3.35,NaN,2.28,...,1.29,0.41,NaN,1.43,27.78,NaN,18.84,0.33,NaN,1.87
574,00ZZ9,None,0.01,0.09,0.14,NaN,NaN,4.02,0.66,5.73,...,0.63,0.96,NaN,NaN,28.06,4.62,39.99,NaN,NaN,1.49
575,0ZZ10,None,0.02,1.04,0.23,NaN,0.26,6.29,2.08,4.33,...,5.41,1.18,NaN,1.38,32.75,10.85,22.54,NaN,NaN,2.10


In [484]:
# Convertir la colonne de jointure en type str pour circo241
circo221['Code circo'] = circo221['Code circo'].astype(str)

# Convertir la colonne de jointure en type str pour circo242
circo222['Code circo'] = circo222['Code circo'].astype(str)
# Ajouter le suffixe aux colonnes de circo242, sauf pour 'Code circonscription législative'
circo222.columns = [col + ' Legi_T222' if col != 'Code circo' else col for col in circo222.columns]

# Ajouter le suffixe aux colonnes de circo241, sauf pour 'Code circonscription législative'
circo221.columns = [col + ' Legi_T122' if col != 'Code circo' else col for col in circo221.columns]

# Fusionner les deux DataFrames sur 'Code circonscription législative'
merged_df = pd.merge(circo221, circo222, on='Code circo', how='left')
merged_df

/tmp/ipykernel_292/727537448.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circo222['Code circo'] = circo222['Code circo'].astype(str)


,Code circo,Nuance élu Legi_T122,% Voix/Ins DIV Legi_T122,% Voix/Ins DVC Legi_T122,% Voix/Ins DVG Legi_T122,% Voix/Ins DXG Legi_T122,% Voix/Ins ECO Legi_T122,% Voix/Ins ENS Legi_T122,% Voix/Ins LR Legi_T122,% Voix/Ins NUP Legi_T122,...,% Voix/Exp DVG Legi_T222,% Voix/Ins RN Legi_T222,% Voix/Exp RN Legi_T222,% Voix/Ins DVC Legi_T222,% Voix/Exp DVC Legi_T222,% Voix/Ins DIV Legi_T222,% Voix/Exp DIV Legi_T222,% Voix/Ins DXG Legi_T222,% Voix/Exp DXG Legi_T222,Nuance élu Legi_T222
0,00011,None,NaN,NaN,NaN,0.45,1.35,9.36,12.30,11.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
1,00012,None,0.27,NaN,NaN,0.42,0.65,12.92,7.20,12.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
2,00013,None,0.00,0.08,NaN,0.34,2.14,13.02,7.82,9.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
3,00014,None,NaN,4.44,0.01,0.32,1.18,7.71,7.59,8.20,...,NaN,23.81,62.27,NaN,NaN,NaN,NaN,NaN,NaN,RN
4,00015,None,0.91,NaN,0.36,0.53,NaN,NaN,4.62,10.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,00ZZ7,None,0.32,1.66,0.55,NaN,0.30,10.06,1.13,7.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS
573,00ZZ8,None,0.43,0.16,0.05,NaN,0.17,3.35,NaN,2.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR
574,00ZZ9,None,0.01,0.09,0.14,NaN,NaN,4.02,0.66,5.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NUP
575,0ZZ10,None,0.02,1.04,0.23,NaN,0.26,6.29,2.08,4.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS


In [485]:
# Fonction pour combiner les nuances et signaler les valeurs non-None
def combine_nuances(row):
    nuance_122 = row.get('Nuance élu Legi_T122', None)
    nuance_222 = row.get('Nuance élu Legi_T222', None)
    
    if pd.notna(nuance_122) and pd.notna(nuance_222):
        print(f"Code circonscription législative {row['Code circo']}: pas none")
        if nuance_122 != nuance_222:
            # Si les deux sont non-None mais différentes, afficher un message d'erreur
            raise ValueError(f"Conflit dans la circonscription législative {row['Code circo']}: '{nuance_122}' vs '{nuance_222}'")
        return nuance_122
    elif pd.notna(nuance_122):
        print(f"Code circonscription législative {row['Code circo']}: Legi_T222 est None")
        return nuance_122
    elif pd.notna(nuance_222):
        print(f"Code circonscription législative {row['Code circo']}: Legi_T122 est None")
        return nuance_222
    else:
        print(f"Code circonscription législative {row['Code circo']}: both none")
        return None

# Appliquer la fonction pour créer la nouvelle colonne
merged_df['Nuance élu Legi22'] = merged_df.apply(combine_nuances, axis=1)
merged_df

Code circonscription législative 00011: Legi_T122 est None
Code circonscription législative 00012: Legi_T122 est None
Code circonscription législative 00013: Legi_T122 est None
Code circonscription législative 00014: Legi_T122 est None
Code circonscription législative 00015: Legi_T122 est None
Code circonscription législative 00021: Legi_T122 est None
Code circonscription législative 00022: Legi_T122 est None
Code circonscription législative 00023: Legi_T122 est None
Code circonscription législative 00024: Legi_T122 est None
Code circonscription législative 00025: Legi_T122 est None
Code circonscription législative 00031: Legi_T122 est None
Code circonscription législative 00032: Legi_T122 est None
Code circonscription législative 00033: Legi_T122 est None
Code circonscription législative 00041: Legi_T122 est None
Code circonscription législative 00042: Legi_T122 est None
Code circonscription législative 00051: Legi_T122 est None
Code circonscription législative 00052: Legi_T122 est No

,Code circo,Nuance élu Legi_T122,% Voix/Ins DIV Legi_T122,% Voix/Ins DVC Legi_T122,% Voix/Ins DVG Legi_T122,% Voix/Ins DXG Legi_T122,% Voix/Ins ECO Legi_T122,% Voix/Ins ENS Legi_T122,% Voix/Ins LR Legi_T122,% Voix/Ins NUP Legi_T122,...,% Voix/Ins RN Legi_T222,% Voix/Exp RN Legi_T222,% Voix/Ins DVC Legi_T222,% Voix/Exp DVC Legi_T222,% Voix/Ins DIV Legi_T222,% Voix/Exp DIV Legi_T222,% Voix/Ins DXG Legi_T222,% Voix/Exp DXG Legi_T222,Nuance élu Legi_T222,Nuance élu Legi22
0,00011,None,NaN,NaN,NaN,0.45,1.35,9.36,12.30,11.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,LR
1,00012,None,0.27,NaN,NaN,0.42,0.65,12.92,7.20,12.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS,ENS
2,00013,None,0.00,0.08,NaN,0.34,2.14,13.02,7.82,9.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS,ENS
3,00014,None,NaN,4.44,0.01,0.32,1.18,7.71,7.59,8.20,...,23.81,62.27,NaN,NaN,NaN,NaN,NaN,NaN,RN,RN
4,00015,None,0.91,NaN,0.36,0.53,NaN,NaN,4.62,10.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,LR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,00ZZ7,None,0.32,1.66,0.55,NaN,0.30,10.06,1.13,7.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS,ENS
573,00ZZ8,None,0.43,0.16,0.05,NaN,0.17,3.35,NaN,2.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,LR
574,00ZZ9,None,0.01,0.09,0.14,NaN,NaN,4.02,0.66,5.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NUP,NUP
575,0ZZ10,None,0.02,1.04,0.23,NaN,0.26,6.29,2.08,4.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENS,ENS


In [486]:
merged_df.columns

Index(['Code circo', 'Nuance élu Legi_T122', '% Voix/Ins DIV Legi_T122',
       '% Voix/Ins DVC Legi_T122', '% Voix/Ins DVG Legi_T122',
       '% Voix/Ins DXG Legi_T122', '% Voix/Ins ECO Legi_T122',
       '% Voix/Ins ENS Legi_T122', '% Voix/Ins LR Legi_T122',
       '% Voix/Ins NUP Legi_T122', '% Voix/Ins RDG Legi_T122',
       '% Voix/Ins REG Legi_T122', '% Voix/Ins RN Legi_T122',
       '% Voix/Exp DIV Legi_T122', '% Voix/Exp DVC Legi_T122',
       '% Voix/Exp DVG Legi_T122', '% Voix/Exp DXG Legi_T122',
       '% Voix/Exp ECO Legi_T122', '% Voix/Exp ENS Legi_T122',
       '% Voix/Exp LR Legi_T122', '% Voix/Exp NUP Legi_T122',
       '% Voix/Exp RDG Legi_T122', '% Voix/Exp REG Legi_T122',
       '% Voix/Exp RN Legi_T122', '% Voix/Ins NUP Legi_T222',
       '% Voix/Exp NUP Legi_T222', '% Voix/Ins ENS Legi_T222',
       '% Voix/Exp ENS Legi_T222', '% Voix/Ins LR Legi_T222',
       '% Voix/Exp LR Legi_T222', '% Voix/Ins REG Legi_T222',
       '% Voix/Exp REG Legi_T222', '% Voix/Ins DVG 